In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

sns.set()

In [2]:
symbol = 'S&P_500'
df = pd.read_csv(f'../datasets/{symbol}.csv')

In [3]:
y = df.trading_signal
X = df[['sma', 'macd', 'stochastic_k', 'stochastic_d', 'rsi', 'william_r']]
X = (X-X.min())/(X.max()-X.min())
train_X, test_X, train_y, test_y = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.25)

In [4]:
model = KNeighborsRegressor(leaf_size= 20, n_neighbors= 8)
model.fit(train_X, train_y)

predictions = model.predict(test_X)
print(f"MAE: {mean_absolute_error(test_y, predictions)}")

MAE: 0.21577862741794965


In [5]:
padding = np.full(train_X.shape[0], np.nan)
prediction_trading_signal = np.concatenate((padding, predictions), axis=None)
df['p_t_s_knn'] = prediction_trading_signal
df.to_csv(f'../datasets/{symbol}.csv')

In [23]:
parameters = {
    'leaf_size': range(20,41),
    'n_neighbors': range(1,40)   
}

grid = GridSearchCV(KNeighborsRegressor(), parameters, cv = 2, n_jobs = 5, verbose=1)
grid.fit(train_X, train_y)

print(f"MAE: {mean_absolute_error(test_y, grid.predict(test_X))}")
print(grid.best_score_, grid.best_params_)

Fitting 2 folds for each of 819 candidates, totalling 1638 fits
MAE: 0.12721148771586419
0.3326347010607735 {'leaf_size': 20, 'n_neighbors': 2}


[Parallel(n_jobs=5)]: Done 1638 out of 1638 | elapsed:    1.8s finished
